In [1]:
import librosa
import os

In [2]:
def getDatafile(file_dir):

	test_list = []
	test_label = [] 
	valide_list = []
	valide_label = []
	train_list = []
	train_label = []


	file_list = []   #liste des files de wav
	dic_list = [] # dictionnaire de base des données
	path_list=os.listdir(file_dir)

	for wav_dir in path_list:
		name = wav_dir.split(sep='.')
		if(len(name) == 1) :
			for wav in os.listdir(file_dir+'/'+wav_dir): 
				name_wav = wav.split(sep='.')
				if(len(name_wav) == 2 and name_wav[1] == 'wav' ):
					file_list.append(wav_dir+'/'+wav)
					if not(wav_dir in dic_list):
						dic_list.append(wav_dir)

	print(len(file_list))
	print(dic_list)

	index = {}
	for i in range(len(dic_list)):
		index[dic_list[i]] = i
	#Creates the reverse table that maps labels names to their index	
		
	print(index)
	f=open("/home/gjx/Desktop/vocale/data_speech_commands_v0.02/testing_list.txt",'r')
	for line in f:
		test_list.append(line.strip('\n'))
		label = line.strip('\n').split(sep='/')[0]
		test_label.append(index[label])


	for line in open("/home/gjx/Desktop/vocale/data_speech_commands_v0.02/validation_list.txt",'r'):
		valide_list.append(line.strip('\n'))
		label = line.strip('\n').split(sep='/')[0]
		valide_label.append(index[label])


	for line in file_list:
		if not (line in test_list ):
			if not (line in valide_list):
				train_list.append(line)
				train_label.append(index[label])

	return index, train_list, train_label, valide_list, valide_label, test_list, test_label


In [3]:
file_dir="/home/gjx/Desktop/vocale/data_speech_commands_v0.02"
index, train_list, train_label, valide_list, valide_label, test_list, test_label=getDatafile(file_dir)

105835
['down', 'learn', 'left', 'forward', 'on', 'marvin', 'one', 'six', 'visual', 'right', 'sheila', 'yes', 'backward', 'eight', 'up', 'bird', 'go', 'wow', 'no', 'bed', 'seven', 'tree', 'cat', 'nine', 'zero', 'four', 'stop', 'follow', '_background_noise_', 'five', 'off', 'dog', 'house', 'two', 'three', 'happy']
{'down': 0, 'learn': 1, 'left': 2, 'forward': 3, 'on': 4, 'marvin': 5, 'one': 6, 'six': 7, 'visual': 8, 'right': 9, 'sheila': 10, 'yes': 11, 'backward': 12, 'eight': 13, 'up': 14, 'bird': 15, 'go': 16, 'wow': 17, 'no': 18, 'bed': 19, 'seven': 20, 'tree': 21, 'cat': 22, 'nine': 23, 'zero': 24, 'four': 25, 'stop': 26, 'follow': 27, '_background_noise_': 28, 'five': 29, 'off': 30, 'dog': 31, 'house': 32, 'two': 33, 'three': 34, 'happy': 35}


In [4]:
# from tqdm import tqdm
# import time
# train_set=[]
# start=time.clock()
# for i in tqdm(range(len(train_list))):
#     y, sr = librosa.load("/home/gjx/Desktop/vocale/data_speech_commands_v0.02/"+train_list[i], sr=None)
#     melspec = librosa.feature.melspectrogram(y, sr, n_fft=1024, hop_length=512, n_mels=128)
#     logmelspec = librosa.power_to_db(melspec)
#     train_set.append(logmelspec)
# end=time.clock()
# print(end-start)
# for i in range(len(train_set)):
#     train_set[i]=train_set[i].reshape(-1)

# import numpy as np
# for i in range(len(train_set)):
#     if train_set[i].size < 4096:
#         t=4096-train_set[i].size
#         a=np.mean(train_set[i])
#         z=np.zeros(t)
#         z.fill(a)
# #         print(tmp.shape)
#         train_set[i]=(np.append(train_set[i],z))
#     elif train_set[i].size > 4096:
#         train_set[i]=train_set[i][:4096]


In [5]:
from tqdm import tqdm
import time
valide_set=[]
start=time.clock()
for i in tqdm(range(len(valide_list))):
    y, sr = librosa.load("/home/gjx/Desktop/vocale/data_speech_commands_v0.02/"+valide_list[i], sr=None)
    melspec = librosa.feature.melspectrogram(y, sr, n_fft=1024, hop_length=512, n_mels=128)
    logmelspec = librosa.power_to_db(melspec)
    valide_set.append(logmelspec)
end=time.clock()
print(end-start)


for i in range(len(valide_set)):
    valide_set[i]=valide_set[i].reshape(-1)

import numpy as np
for i in range(len(valide_set)):
    if valide_set[i].size < 4096:
        t=4096-valide_set[i].size
        a=np.mean(valide_set[i])
        z=np.zeros(t)
        z.fill(a)
#         print(tmp.shape)
        valide_set[i]=(np.append(valide_set[i],z))
    elif valide_set[i].size > 4096:
        valide_set[i]=valide_set[i][:4096]

100%|██████████| 9981/9981 [02:31<00:00, 65.85it/s]


1088.557173


In [6]:
from sklearn import datasets, neighbors, linear_model
knn = neighbors.KNeighborsClassifier(n_neighbors=3)

knn.fit(valide_set,valide_label)


KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=3, p=2,
           weights='uniform')

In [7]:
start=time.clock()
print(knn.score(valide_set,valide_label))
end=time.clock()
print(end-start)

0.6567478208596333
692.0466549999999


In [8]:
import gc
del valide_set
gc.collect()

6

In [9]:
from tqdm import tqdm
import time
test_set=[]
start=time.clock()
for i in tqdm(range(len(test_list))):
    y, sr = librosa.load("/home/gjx/Desktop/vocale/data_speech_commands_v0.02/"+test_list[i], sr=None)
    melspec = librosa.feature.melspectrogram(y, sr, n_fft=1024, hop_length=512, n_mels=128)
    logmelspec = librosa.power_to_db(melspec)
    test_set.append(logmelspec)
end=time.clock()
print(end-start)


for i in range(len(test_set)):
    test_set[i]=test_set[i].reshape(-1)

import numpy as np
for i in range(len(test_set)):
    if test_set[i].size < 4096:
        t=4096-test_set[i].size
        a=np.mean(test_set[i])
        z=np.zeros(t)
        z.fill(a)
#         print(tmp.shape)
        test_set[i]=(np.append(test_set[i],z))
    elif test_set[i].size > 4096:
        test_set[i]=test_set[i][:4096]

100%|██████████| 11005/11005 [01:44<00:00, 104.88it/s]


778.7667550000001


In [10]:
start=time.clock()
print(knn.score(test_set,test_label))
end=time.clock()
print(end-start)

0.35374829622898685
673.6409660000004
